# Bayesian inference with pySam
pySam provides a simple implementation of Monte Carlo techniques such as importance sampling and sequential MC for static models.

The package contains a base class for Bayesian models `pysam.Model`

```python
class Model:
    
    def __init__(self): raise NotImplementedError
    def LogLikelihood(self,X): raise NotImplementedError
    def LogPrior(self,X): raise NotImplementedError
    def rPrior(self,X): raise NotImplementedError
    def Density(self,X): raise NotImplementedError
```
where `LogLikelihood`, `LogPrior` and `Density` are the data log likelihood, the log of the prior distribution of the model parameters, and `Density` is the probabilistic model. The user implementation of these functions must support vectorization, i.e. `X` can be a numpy array with set of parameters arranged by rows.

The main class implementing sampling methods is `pysam.Inference`. An instance of `pysam.Inference` is specified by a Model object

```python
def __init__(self,model):
        self.model = model
```

## Importance sampling 
Posterior averages can be obtained by using weighted samples obtained from an importance distribution used to approximate the posterior density. Importance weights are obtained as the ratio between the unnormalized posterior and the importance densities.

## Sequential Monte Carlo 
In pySam we implemented a tempered likelihood scheme to obtain a particle approximation of the posterior density. The method `pysam.SMC` corresponds to the following algorithm[1]

1. Initialization 
     * set $n=0$ 
     * draw $N$ particles corresponding to independent model parameters $\theta_{1:N}$ according to their prior distributions
     * Set importance weights to $1/N$ for all particles
     * Define a tempered protocol $h=\lbrace h_0=0,h_1,\cdots,h_{P-1},h_P=1\rbrace$
     * Iterate steps 2, 3 and 4
2. Resampling
     * Calculate the effective sample size (ESS)
     $$ESS = \left(\sum_k (W^{(k)})^2\right)^{-1}$$
     * when $ESS<N/2$ resample all particles and set their weights to $1/N$
3. Reweighting 
     * Reweight all particles using an incremental log weight of 
      $\log\tilde w = (h_{n+1}-h_{n}) \log P(data|\theta)$
     * recalculate and normalize all weights 
4. Move particles according to a Metropolis-Hastings kernel. 

## References
[1] Del Moral, Pierre, Arnaud Doucet, and Ajay Jasra. "Sequential monte carlo samplers." Journal of the Royal Statistical Society: Series B (Statistical Methodology) 68.3 (2006): 411-436.